## Welcome to Week 4, Day 4

This is the start of an AWESOME project! Really simple and very effective.

In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

In [2]:
load_dotenv(override=True)

True

### Asynchronous LangGraph

To run a tool:  
Sync: `tool.run(inputs)`  
Async: `await tool.arun(inputs)`

To invoke the graph:  
Sync: `graph.invoke(state)`  
Async: `await graph.ainvoke(state)`

In [3]:
class State(TypedDict):
    
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [4]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})

tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
    )

### Q. whay is `playwright`?

- Automate real browsers from Python: Playwright lets you control a real browser (Chromium, Firefox, WebKit) using Python to navigate pages, click elements, fill forms, and extract content after JavaScript has fully run.

- Headless mode for speed and automation: In headless mode, the browser runs invisibly in the background, making it ideal for CI pipelines, large-scale scraping, automated testing, and agent workflows where performance and resource usage matter.

- Headful mode for debugging and interaction: In headful mode, you can see the browser window, watch actions happen in real time, and manually interact if needed—this is especially useful for debugging selectors, timing issues, and complex UI behavior.

- Reliable handling of modern web apps: Playwright handles async JavaScript, dynamic DOM updates, and SPA frameworks out of the box, which is critical when Python scripts need accurate page state instead of raw HTML.

- Async-first Python integration: Playwright works naturally with Python’s async/await, allowing efficient parallel browser actions and smooth integration with async frameworks, agents, and tool-based workflows.

## Extra installation step - if you don't have Node and Playwright on your computer

Next, you need to install NodeJS and Playwright on your computer if you don't already have them. Please see instructions here:

[Node and Playwright setup](../setup/SETUP-node.md)

In [5]:
# Introducing nest_asyncio
# Python async code only allows for one "event loop" processing aynchronous events.
# The `nest_asyncio` library patches this, and is used for special situations, if you need to run a nested event loop.

import nest_asyncio
nest_asyncio.apply()

### The LangChain community

One of the remarkable things about LangChain is the rich community around it.

Check this out:


In [6]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

# If you get a NotImplementedError here or later, see the Heads Up at the top of the notebook

async_browser =  create_async_playwright_browser(headless=False)  # headful mode
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

In [7]:
for tool in tools:
    print('---')
    print(f"{tool.name}={tool}")

---
click_element=async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/alin/.cache/ms-playwright/chromium-1169/chrome-linux/chrome> version=136.0.7103.25>
---
navigate_browser=async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/alin/.cache/ms-playwright/chromium-1169/chrome-linux/chrome> version=136.0.7103.25>
---
previous_webpage=async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/alin/.cache/ms-playwright/chromium-1169/chrome-linux/chrome> version=136.0.7103.25>
---
extract_text=async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/alin/.cache/ms-playwright/chromium-1169/chrome-linux/chrome> version=136.0.7103.25>
---
extract_hyperlinks=async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/alin/.cache/ms-playwright/chromium-1169/chrome-linux/chrome> version=136.0.7103.25>
---
get_elements=async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/a

In [8]:
tool_dict = {tool.name:tool for tool in tools}

navigate_tool = tool_dict.get("navigate_browser")
extract_text_tool = tool_dict.get("extract_text")

    
await navigate_tool.arun({"url": "https://www.cnn.com"})
text = await extract_text_tool.arun({})

In [ ]:
import textwrap
print(textwrap.fill(text))

In [17]:
all_tools = tools + [tool_push]

In [18]:

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(all_tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


In [ ]:

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=all_tools))
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "10"}}

async def chat(user_input: str, history):
    result = await graph.ainvoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()